Goal is to predict credit card fraud (prioritizing sensitivity over specificity). From the Kaggle [description](https://www.kaggle.com/mlg-ulb/creditcardfraud):

The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

Given the class imbalance ratio, we recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful for unbalanced classification.

The dataset has been collected and analysed during a research collaboration of Worldline and the Machine Learning Group (http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection. More details on current and past projects on related topics are available on http://mlg.ulb.ac.be/BruFence and http://mlg.ulb.ac.be/ARTML

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
from sklearn.model_selection import cross_val_score
%matplotlib inline

fraud = pd.read_csv("creditcard.csv")

In [2]:
fraud.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
fraud['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

Before we do anything, let's remove time and normalize the class variable.

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

feat_to_scale = fraud.drop(['Time','Class'],axis=1)

scaled_feat = scaler.fit_transform(feat_to_scale)

X = fraud.drop(['Time','Class'],axis=1)
X_scaled = scaled_feat
Y = fraud['Class']

Trying a nearest neighbors model.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

knn = KNeighborsClassifier(n_neighbors=5, weights='distance')

cross_val_score(knn,X_scaled,Y,scoring='recall')
print('Cross validation recall scores: /n')
print(score)
print('Avg Score: {}'.format(score.mean()))
print('Score Std: {}'.format(score.std()))

Random Forest...

In [7]:
from sklearn import ensemble
rfc = ensemble.RandomForestClassifier()

score = cross_val_score(rfc,X,Y,scoring='recall')
print('Cross validation recall scores: /n')
print(score)
print('Avg Score: {}'.format(score.mean()))
print('Score Std: {}'.format(score.std()))

Cross validation recall scores: /n
[0.83536585 0.69512195 0.67073171]
Avg Score: 0.733739837398374
Score Std: 0.07254702559989733


Ridge Regression...

In [ ]:
from sklearn import linear_model
ridgereg = linear_model.LogisticRegression(penalty='l2',c=100)

score = cross_val_score(ridgereg,X,Y,scoring='recall')
print('Cross validation recall scores: /n')
print(score)
print('Avg Score: {}'.format(score.mean()))
print('Score Std: {}'.format(score.std()))

Lasso Regression...

In [9]:
from sklearn import linear_model
lassoreg = linear_model.LogisticRegression(penalty='l1')

score = cross_val_score(lassoreg,X,Y,scoring='recall')
print('Cross validation recall scores: /n')
print(score)
print('Avg Score: {}'.format(score.mean()))
print('Score Std: {}'.format(score.std()))

Cross validation recall scores: /n
[0.82926829 0.48780488 0.4695122 ]
Avg Score: 0.5955284552845528
Score Std: 0.16544765386153432


Gradient Boosted...

In [12]:
params = {'n_estimators': 100,
          'max_depth': 10,
          'loss': 'deviance',
         #'learning_rate':0.08
         }

gradient = ensemble.GradientBoostingClassifier(**params)

score = cross_val_score(gradient,X,Y,scoring='recall')
print('Cross validation recall scores: /n')
print(score)
print('Avg Score: {}'.format(score.mean()))
print('Score Std: {}'.format(score.std()))

Cross validation recall scores: /n
[0.84146341 0.69512195 0.69512195]
Avg Score: 0.7439024390243901
Score Std: 0.06898602743283391


In [17]:
params = {'n_estimators': [1,1000],'max_depth':np.arange(1,10)}

In [19]:
from sklearn.model_selection import GridSearchCV

gradient = ensemble.GradientBoostingClassifier()

clf = GridSearchCV(gradient, param_grid)

clf.fit(X,Y)

KeyboardInterrupt: 

Support vector machine...

In [ ]:
from sklearn.svm import svm
svr = svm.svc()

score = cross_val_score(svr,X,Y,scoring='recall')
print('Cross validation recall scores: /n')
print(score)
print('Avg Score: {}'.format(score.mean()))
print('Score Std: {}'.format(score.std()))

Naive Bayes...

In [5]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()

score = cross_val_score(bnb,X,Y,scoring='recall')
print('Cross validation recall scores: /n')
print(score)
print('Avg Score: {}'.format(score.mean()))
print('Score Std: {}'.format(score.std()))

Cross validation recall scores: /n
[0.73170732 0.62195122 0.56097561]
Avg Score: 0.6382113821138211
Score Std: 0.07064287478448276
